<a href="https://colab.research.google.com/github/mrugeles/np-exercises/blob/master/Modelo_Regresion_Covid_19_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sodapy

# Modelo de regresión sobre casos en Colombia de Covid-19

Este es un ejemplo de un modelo de regresión con un dataset muy limitado de apenas 14 registros (Marzo 19 del 2020) disponibles en la página del [Instituto Nacional de Salud](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx).

Este notebook se ha creado con fines educativos para demostrar cómo realizar un modelo de regresión polinomial.

Hay que tener en cuenta que la evolución de la infermedad con el tiempo define una curva logística y no una exponencial, esto es, la curva alcanza un punto de inflexión para luego tener una tendencia hacia una asintota horizonal.

In [ ]:
import pandas as pd
import numpy as np
import operator
import datetime
from scipy.signal import find_peaks

import plotly.graph_objects as go

from sklearn.model_selection import train_test_split

endpoint = 'https://www.datos.gov.co/resource/gt2j-8ykr.json'


In [ ]:
from sodapy import Socrata

client = Socrata("www.datos.gov.co", None)
results = client.get_all("gt2j-8ykr")
results_df = pd.DataFrame.from_records(results)
results_df.tail()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,fecha_de_muerte,nombre_grupo_etnico
97841,97882,2020-06-25T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,41,M,En estudio,Leve,NaN,2020-06-23T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,11,NaN,NaN,NaN,NaN
97842,97883,2020-06-25T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,20,F,En estudio,Leve,NaN,2020-06-24T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,11,NaN,NaN,NaN,NaN
97843,97884,2020-06-25T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,34,M,En estudio,Leve,NaN,2020-06-24T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,11,NaN,NaN,NaN,NaN
97844,97885,2020-06-16T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,67,M,En estudio,Asintomático,NaN,Asintomático,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,11,NaN,NaN,NaN,NaN
97845,97886,2020-06-23T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,36,M,En estudio,Leve,NaN,2020-06-21T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,11,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

def to_polinomyal(x):
    polynomial_features= PolynomialFeatures(degree=3)
    x = x[:, np.newaxis]

    return polynomial_features.fit_transform(x)

def build_model(x, y):
    x = to_polinomyal(x)
    y = to_polinomyal(y)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=300)

    model = LinearRegression()
    return model.fit(X_train, y_train)

def get_predictions(model, x_predic): 
    x_predic = to_polinomyal(x_predic)
    y_predic = model.predict(x_predic)

    predictions = pd.DataFrame(columns=['x', 'y'])
    predictions['x'] = x_predic[:, 1]
    predictions['y'] = y_predic[:, 1]

    return predictions

In [ ]:
dataset = results_df
dataset['fecha_de_notificaci_n'] = pd.to_datetime(dataset['fecha_de_notificaci_n'])
dataset.sort_values(by = 'fecha_de_notificaci_n', inplace=True)
dataset.tail()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,fecha_de_muerte,nombre_grupo_etnico
96177,96218,2020-06-30,54810,Tibú,Norte de Santander,Casa,44,M,En estudio,Leve,NaN,2020-06-28T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,54,NaN,NaN,NaN,NaN
95374,95415,2020-06-30,63130,Calarcá,Quindío,Casa,71,M,En estudio,Leve,NaN,2020-06-26T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,63,NaN,NaN,NaN,NaN
97136,97177,2020-06-30,63130,Calarcá,Quindío,Casa,38,M,En estudio,Leve,NaN,2020-06-17T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,63,NaN,NaN,NaN,NaN
96591,96632,2020-06-30,05266,Envigado,Antioquia,Casa,27,F,En estudio,Leve,NaN,2020-06-20T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
97027,97068,2020-06-30,05425,Maceo,Antioquia,Casa,50,M,En estudio,Leve,NaN,2020-06-22T00:00:00.000,2020-06-30T00:00:00.000,NaN,2020-06-30T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN


In [ ]:
cases_by_day = dataset.groupby('fecha_de_notificaci_n').count()
cases_by_day.reset_index(inplace=True)
cases_by_day.rename(columns={'fecha_de_notificaci_n': 'date', 'id_de_caso':'cases'}, inplace=True)
cases_by_day = cases_by_day[['date', 'cases']]
cases_by_day.head()

,date,cases
0,2020-03-02,1
1,2020-03-06,1
2,2020-03-07,1
3,2020-03-08,2
4,2020-03-09,3


# Cases by day

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['date'], y=cases_by_day['cases'], mode='lines+markers', name='Cases by day'))
fig.show()

In [ ]:
cases_by_day['x'] = np.arange(0, cases_by_day.shape[0])
cases_by_day['y'] = cases_by_day['cases'].cumsum()
cases_by_day.tail()

,date,cases,x,y
113,2020-06-26,1376,113,96831
114,2020-06-27,669,114,97500
115,2020-06-28,177,115,97677
116,2020-06-29,163,116,97840
117,2020-06-30,6,117,97846


# Aggregated cases

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['x'], y=cases_by_day['y'], mode='lines+markers', name='Acumulated cases'))
fig.show()

In [ ]:
cases_by_day.describe()

,cases,x,y
count,118.000000,118.000000,118.000000
mean,829.203390,58.500000,25283.754237
std,893.565953,34.207699,30418.230222
min,1.000000,0.000000,1.000000
25%,159.000000,29.250000,2495.750000
50%,428.500000,58.500000,9935.000000
75%,1374.750000,87.750000,40051.000000
max,3357.000000,117.000000,97846.000000


In [ ]:
prediction_days = 300

In [ ]:
prediction_days = 300
train_df = cases_by_day[:100]
model = build_model(train_df['x'], train_df['y'])
predictions = get_predictions(model, np.arange(0, prediction_days))
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['x'], y=cases_by_day['y'], mode='lines+markers', name='Confirmed cases'))
fig.add_trace(go.Scatter(x=predictions['x'], y=predictions['y'], mode='lines', name='Projection'))

fig.update_layout(
    title_text="Confirmed cases vs Predictions"
)
fig.show()

## Get the inflection point for the logistic function

In [ ]:
steps = predictions['y']
growth_factor = np.array([x / steps[i - 1] for i, x in enumerate(steps) if i > 0])
growth_factor = np.concatenate((np.array([0]), growth_factor))
predictions['growth_factor'] = growth_factor
peaks, _ = find_peaks(predictions['growth_factor'], height=0)
fig = go.Figure()
fig.add_trace(go.Scatter(x=predictions['x'], y=predictions['growth_factor'], mode='lines', name=''))

fig.update_layout(
    title_text="Growth factor"
)
fig.show()
print(peaks)

[ 1  7 49]


In [ ]:
inflection_row = predictions[predictions['growth_factor'] > 1.02].tail(1)
inflection_idx = inflection_row.index[0]
inflection_x = inflection_row['x'].max()
inflection_y = inflection_row['y'].max()
L = inflection_y*2

print(f'inflection_idx: {inflection_idx}')
print(f'inflection_x: {inflection_x}')
print(f'inflection_y: {inflection_y}')
print(f'Inflection date: {inflection_y}')
print(f'L: {L}')

inflection_idx: 169
inflection_x: 169.0
inflection_y: 388626.4008827899
Inflection date: 388626.4008827899
L: 777252.8017655798


In [ ]:
pd.date_range(start=cases_by_day['date'].min(), periods=prediction_days)[inflection_idx]

Timestamp('2020-08-18 00:00:00', freq='D')

# Exponential range for logistic function

In [ ]:
pred = predictions.iloc[:inflection_idx+1]
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred['x'], y=pred['y'], line=dict(dash='dash'), name = 'Projection'))
fig.add_trace(go.Scatter(x=cases_by_day['x'], y=cases_by_day['y'], mode='lines+markers', name='Confirmed cases'))


fig.update_layout(
  title_text="Exponential range for logistic function"
)

fig.show()

## Build logistic function

In [ ]:
cases_by_day['date'].min()

Timestamp('2020-03-02 00:00:00')

In [ ]:
predictions_inv = predictions[:inflection_idx + 1][::-1].copy()
predictions_b = predictions_inv.copy()

predictions_b['x'] = np.arange(inflection_idx, inflection_idx*2+1)
predictions_b['y'] = predictions_b['y'].apply(lambda y: L - y)

full_predictions = predictions[:inflection_idx]
full_predictions = full_predictions.append(predictions_b)

full_predictions.loc[:,'date'] = pd.date_range(start=cases_by_day['date'].min(), periods=full_predictions.shape[0])

fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['date'], y=cases_by_day['y'], mode='lines+markers', line=dict(width=1), name='Confirmed cases'))
fig.add_trace(go.Scatter(x=full_predictions['date'], y=full_predictions['y'], line=dict(dash='dash'), name = 'Projection'))

fig.update_layout(
    title_text="Confirmed cases vs Predictions"
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['date'], y=cases_by_day['y'], mode='lines+markers', line=dict(width=1), name='Confirmed cases'))

fig.update_layout(
    title_text="Acumulado casos confirmados"
)

fig.show()

In [ ]:
cases_by_day.head()

,date,cases,x,y
0,2020-03-02,1,0,1
1,2020-03-06,1,1,2
2,2020-03-07,1,2,3
3,2020-03-08,2,3,5
4,2020-03-09,3,4,8


In [ ]:
from scipy.optimize import curve_fit

def logifunc(x,A,x0,k,off):
    return A / (1 + np.exp(-k*(x-x0)))+off

popt, pcov = curve_fit(logifunc, full_predictions['x'], full_predictions['y'], p0=[0,10000,0.1,-222])
popt

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated



array([0.00000000e+00, 1.00000000e+04, 1.00000000e-01, 3.88626401e+05])

In [ ]:

a = inflection_idx
b = 300
c = L
print(a, b, c)
fig = go.Figure()
#fig.add_trace(go.Scatter(x=cases_by_day['x'], y=cases_by_day['y'], mode='lines+markers', line=dict(width=1), name='Confirmed cases'))
#fig.add_trace(go.Scatter(x=full_predictions['x'], y=full_predictions['y'], line=dict(dash='dash'), name = 'Projection'))
fig.add_trace(go.Scatter(x=full_predictions['x'], y=logifunc(full_predictions['x'], *popt), line=dict(dash='dash'), name = 'Projection'))

fig.update_layout(
    title_text="Confirmed cases vs Predictions"
)

fig.show()

169 300 777252.8017655798


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

overflow encountered in exp



In [ ]:
def get_proyections(days, prediction_days_range):
  train_df = cases_by_day[:days]
  model = build_model(train_df['x'], train_df['y'])
  return get_predictions(model, prediction_days_range)


In [ ]:
models_predictions = pd.DataFrame(columns=['model', 'x', 'y'])
for n in range(5, cases_by_day.shape[0] + 1, 5):
  df = get_proyections(n, np.arange(0, prediction_days))
  df['model'] = n
  models_predictions = pd.concat([models_predictions, df], ignore_index=True)

models_predictions.head()

,model,x,y
0,5,0.0,1.000000
1,5,1.0,2.000000
2,5,2.0,3.166667
3,5,3.0,5.000000
4,5,4.0,8.000000


In [ ]:
import plotly.express as px

fig = go.Figure()
for predictions in models_predictions.groupby('model'):
  fig.add_trace(go.Scatter(x=predictions[1]['x'], y=predictions[1]['y'], mode='lines+markers', name=f'Model {predictions[0]}'))
  
fig.show()